In [1]:
import math
from collections import Counter
from matplotlib import pyplot as plt
import random
import pandas as pd
import os
import sys
import csv
import dateutil.parser
from packages.Statistics import correlation
from collections import Counter
import numpy as np
from collections import Counter, defaultdict
import re

In [2]:
raw_data = []
raw_data = pd.read_csv("datasets/true_car_listings.csv")
# Credit: https://www.kaggle.com/jpayne/852k-used-car-listings/version/2#true_car_listings.csv

In [3]:
raw_data.head()

,Price,Year,Mileage,City,State,Vin,Make,Model
0,8995,2014,35725,El Paso,TX,19VDE2E53EE000083,Acura,ILX6-Speed
1,10888,2013,19606,Long Island City,NY,19VDE1F52DE012636,Acura,ILX5-Speed
2,8995,2013,48851,El Paso,TX,19VDE2E52DE000025,Acura,ILX6-Speed
3,10999,2014,39922,Windsor,CO,19VDE1F71EE003817,Acura,ILX5-Speed
4,14799,2016,22142,Lindon,UT,19UDE2F32GA001284,Acura,ILXAutomatic


In [4]:
top_brands = pd.read_csv("datasets/us_top_18_brands_2018.csv")

In [5]:
top_models = pd.read_csv("datasets/popular_models.csv")

In [6]:
top_models.head()

,Model,Month,Month LY,Change,YTD,YTD LY,YTD Change
0,Toyota Camry,"29,093","43,331",-32.86,"343,439","387,081",-11.27
1,Honda Civic,"26,384","31,406",-15.99,"325,760","377,286",-13.66
2,Toyota Corolla,"25,439","19,969",27.39,"303,732","329,196",-7.74
3,Honda Accord,"28,627","22,115",29.45,"291,071","322,655",-9.79
4,Nissan Sentra,"17,567","16,834",4.35,"213,046","218,451",-2.47


In [7]:
brand_model = [[top_models['Model'][i].split(" ")[0],
               top_models['Model'][i].split(" ")[1]]
               for i in range(len(top_models))]

In [8]:
brand_model_make = [brand_model[i][0]
                   for i in range(len(brand_model))]
brand_model_model = [brand_model[i][1]
                   for i in range(len(brand_model))]

In [9]:
raw_data = raw_data.loc[raw_data['Make'].isin(list(set(brand_model_make)))]

In [10]:
raw_data.index = range(len(raw_data))

In [11]:
raw_data.head()

,Price,Year,Mileage,City,State,Vin,Make,Model
0,8995,2014,35725,El Paso,TX,19VDE2E53EE000083,Acura,ILX6-Speed
1,10888,2013,19606,Long Island City,NY,19VDE1F52DE012636,Acura,ILX5-Speed
2,8995,2013,48851,El Paso,TX,19VDE2E52DE000025,Acura,ILX6-Speed
3,10999,2014,39922,Windsor,CO,19VDE1F71EE003817,Acura,ILX5-Speed
4,14799,2016,22142,Lindon,UT,19UDE2F32GA001284,Acura,ILXAutomatic


In [12]:
categorized_model = defaultdict(list)
for i,j in brand_model:
    categorized_model[i].append(j)

In [13]:
categorized_model.keys()

dict_keys(['Toyota', 'Honda', 'Nissan', 'Hyundai', 'Ford', 'Chevrolet', 'Tesla', 'Kia', 'Volkswagen', 'Dodge', 'Subaru', 'Mazda', 'Mercedes-Benz', 'Lexus', 'Chrysler', 'BMW', 'Infiniti', 'Audi', 'Acura', 'Mini', 'Mitsubishi', 'Lincoln', 'Cadillac', 'Buick', 'Alfa', 'Volvo', 'Porsche', 'Genesis', 'Fiat', 'Jaguar', 'Smart'])

In [14]:
j = 0
new_data = pd.DataFrame()
for i in range(len(raw_data)-1):
    if raw_data['Make'][i] != raw_data['Make'][i+1] or i == len(raw_data)-2:
        new_data = new_data.append(raw_data[j:i].sort_values(by=['Model']))
        j = i
new_data.index = range(len(new_data))

In [15]:
new_data['General Model'] = ''
new_data.head()

,Price,Year,Mileage,City,State,Vin,Make,Model,General Model
0,4995,1997,121462,Sneads Ferry,NC,19UYA2246VL003892,Acura,CLAutomatic,
1,5500,2001,87625,Lynnwood,WA,19UYA42491A036676,Acura,CLAutomatic,
2,2981,1999,224765,Oklahoma City,OK,19UYA2254XL001148,Acura,CLAutomatic,
3,4495,1998,93969,Danbury,CT,19UYA2251WL010730,Acura,CLAutomatic,
4,4995,2001,139903,Indianapolis,IN,19UYA42421A021467,Acura,CLAutomatic,


In [16]:
len(new_data)

728779

In [17]:
j = 0
for i in range(len(new_data)-1):
    if new_data['Model'][i] != new_data['Model'][i+1] or i == len(new_data)-2:
        count = 0
        for carmodel in categorized_model[new_data['Make'][i]]:
            count = count + 1
            if carmodel in new_data['Model'][i]:
                new_data.loc[np.arange(j,i+1),'General Model'] = carmodel
                break
            elif count == len(categorized_model[new_data['Make'][i]]):
                new_data = new_data.drop(np.arange(j,i+1))
        
        j = i+1

In [18]:
new_data.index = range(len(new_data))

In [36]:
new_data.head()

,Price,Year,Mileage,City,State,Vin,Make,Model,General Model,New Car Price
0,15490,2015,40131,Kennesaw,GA,19VDE1F34FE007892,Acura,ILX4dr,ILX,25900
1,18362,2015,14809,VIRGINIA BEACH,VA,19VDE1F33FE003882,Acura,ILX5-Speed,ILX,25900
2,17288,2014,35613,Arlington,TX,19VDE1F7XEE000401,Acura,ILX5-Speed,ILX,25900
3,17995,2015,19896,Bethlehem,PA,19VDE1F39FE000856,Acura,ILX5-Speed,ILX,25900
4,17350,2015,40840,Rosedale,MD,19VDE1F59FE003371,Acura,ILX5-Speed,ILX,25900


In [37]:
sample_data = pd.DataFrame()

In [38]:
j = 0

for i in range(len(new_data)-1):
    if new_data['General Model'][i] != new_data['General Model'][i+1] or i == len(new_data)-2:
        if (i-j) > 18:
            sample_data = sample_data.append(new_data[j:i].sample(n=20))
        j = i+1

In [41]:
sample_data.index = range(len(sample_data))
sample_data.head()

,Price,Year,Mileage,City,State,Vin,Make,Model,General Model,New Car Price
0,21747,2016,20038,Fort Myers,FL,19UDE2F77GA006421,Acura,ILXAutomatic,ILX,25900
1,13995,2014,67261,Nashville,TN,19VDE1F35EE009147,Acura,ILX5-Speed,ILX,25900
2,14999,2015,42989,Duluth,GA,19VDE1F37FE000614,Acura,ILX5-Speed,ILX,25900
3,19000,2015,23779,Verona,NJ,19VDE1F36FE008882,Acura,ILX5-Speed,ILX,25900
4,21997,2016,21206,Ramsey,NJ,19UDE2F36GA020212,Acura,ILXAutomatic,ILX,25900


In [43]:
from bs4 import BeautifulSoup
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

driver = webdriver.Chrome(ChromeDriverManager().install())
import requests


Checking for win32 chromedriver:2.46 in cache
Driver found in C:\Users\Danniel\.wdm\chromedriver\2.46\win32/chromedriver.exe


In [51]:
def google_car_price(brand,model):
    driver.get('https://www.google.com/search?q=+'+brand+ '+' + model + '+2019+price')
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    if soup.find(id = 'recaptcha'):
        input("Resolve Captcha then press Enter to continue...")
           
    while True:
        try:
            search = soup.find(id = "search")
            price_pedding = search.find(class_="Z0LcW").text
            return (re.sub(r'\D', "", price_pedding))
        except(AttributeError):
            return 'NaN'

In [52]:
google_car_price('acura','ILX')

'25900'

In [53]:
sample_data['New Car Price'] = ''

In [54]:
j = 0
for i in range(len(sample_data)-1):
    if sample_data['General Model'][i] != sample_data['General Model'][i+1]: 
        new_car_price = google_car_price(sample_data['Make'][i],sample_data['General Model'][i])
        sample_data.loc[np.arange(j,i+1),'New Car Price'] = new_car_price
        j = i+1
    if i == len(sample_data)-2:
        new_car_price = google_car_price(sample_data['Make'][i],sample_data['General Model'][i])
        sample_data.loc[np.arange(j,i+2),'New Car Price'] = new_car_price

Resolve Captcha then press Enter to continue...


In [70]:
sample_data[2318:2320]

,Price,Year,Mileage,City,State,Vin,Make,Model,General Model,New Car Price
2318,17997,2016,224,Tempe,AZ,WVWKP7AU7GW915465,Volkswagen,e-GolfSE,Golf,21845
2319,17995,2016,5419,Colma,CA,WVWKP7AU4GW907565,Volkswagen,e-GolfSE,Golf,21845


In [68]:
sample_data['New Car Price'][2318]

'21845'

In [71]:
sample_data.to_csv("datasets/Car_model_price_modified.csv")